# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [31]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import json

# Import API key
from api_keys import geoapify_key

In [32]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,manggar,-2.8833,108.2667,27.19,81,32,1.31,ID,1702305491
1,1,beau vallon,-20.4189,57.6953,27.13,74,40,3.60,MU,1702305492
2,2,nova sintra,14.8667,-24.7167,23.50,61,1,7.23,CV,1702305305
3,3,albany,42.6001,-73.9662,2.54,90,100,1.79,US,1702305292
4,4,papao,-17.7500,-149.5500,23.82,80,61,10.59,PF,1702305492


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [33]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    'Lng',
    'Lat',
    geo= True,
    tiles= 'OSM',
    size = 'Humidity',
    color = 'City'
    )

# Display the map
map_plot_1

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [35]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df = city_data_df.loc[(city_data_df['Max Temp']> 18) & (city_data_df['Max Temp'] < 30)]
city_data_df = city_data_df.loc[(city_data_df['Wind Speed'] < 4.5)]
city_data_df = city_data_df.loc[(city_data_df['Cloudiness'] ==0)]


# Drop any rows with null values
city_data_df.dropna(axis = 0)

# Display sample data
city_data_df.shape

(20, 10)

### Step 3: Create a new DataFrame called `hotel_df`.

In [36]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = city_data_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel Name
20,zaragoza,ES,41.6561,-0.8773,50,
55,patamundai,IN,20.5667,86.5667,42,
61,haiku-pauwela,US,20.9219,-156.3051,88,
96,forster,AU,-32.1667,152.5167,96,
101,la manga del mar menor,ES,37.6413,-0.7165,69,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [37]:
# Set parameters to search for a hotel
radius = 10000
limit = 100
params = {
    "limit":limit,
    "apiKey":geoapify_key, 
    "categories": "accommodation.hotel"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    latitude = row['Lat']
    longitude = row['Lng']
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
zaragoza - nearest hotel: Hotel Pilar Plaza
patamundai - nearest hotel: No hotel found
haiku-pauwela - nearest hotel: Inn At Mama's Fish House
forster - nearest hotel: No hotel found
la manga del mar menor - nearest hotel: Hotel Entremares
hermanus - nearest hotel: Aloe guest house
plettenberg bay - nearest hotel: Milkwood Manor
pisco - nearest hotel: La Portada
santander - nearest hotel: Los Caracoles
dogonbadan - nearest hotel: هتل پاسارگاد
howrah - nearest hotel: Sun India Guest House
el bayadh - nearest hotel: No hotel found
garachico - nearest hotel: Residencia El Drago
las palmas - nearest hotel: Era de La Corte
kruisfontein - nearest hotel: No hotel found
holualoa - nearest hotel: Kona Hotel
salalah - nearest hotel: Muscat International Hotel
mao - nearest hotel: Hevresac
alamos - nearest hotel: No hotel found
crucecita - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
20,zaragoza,ES,41.6561,-0.8773,50,Hotel Pilar Plaza
55,patamundai,IN,20.5667,86.5667,42,No hotel found
61,haiku-pauwela,US,20.9219,-156.3051,88,Inn At Mama's Fish House
96,forster,AU,-32.1667,152.5167,96,No hotel found
101,la manga del mar menor,ES,37.6413,-0.7165,69,Hotel Entremares
127,hermanus,ZA,-34.4187,19.2345,62,Aloe guest house
141,plettenberg bay,ZA,-34.0527,23.3716,60,Milkwood Manor
169,pisco,PE,-13.7000,-76.2167,78,La Portada
203,santander,ES,43.4647,-3.8044,73,Los Caracoles
204,dogonbadan,IR,30.3586,50.7981,39,هتل پاسارگاد


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [39]:
%%capture --no-display

# Configure the map plot
map_plot_2 = hotel_df.hvplot.points(
    'Lng',
    'Lat',
    geo= True,
    tiles= 'OSM',
    size = 'Humidity',
    color = 'City',
    hover_cols = ['Hotel Name', 'Country']
    )

#, hotel_df['Country']

# Display the map
map_plot_2

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)